In [32]:
!git clone https://github.com/labib-c/toronto-public-tennis-courts-notebook.git

Cloning into 'toronto-public-tennis-courts-notebook'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 9 (delta 0), reused 6 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), 211.98 KiB | 2.49 MiB/s, done.


In [1]:
# prompt: open tennis_courts_toronto csv with pandas

import pandas as pd

# Assuming the file is in the current directory or specified path
df = pd.read_csv('tennis_courts_toronto.csv')

# Display the first few rows to confirm it's loaded
print(df.head())

                  Location                                  Club Public Hours  \
0    Agincourt Tennis Club  Sundays 2 p.m. - 4 p.m.Tuesdays 4 p.m. - 6 p.m...   
1  Bridlebrook Tennis Club                                                NaN   
2      Albion Gardens Park                                                NaN   
3            Amesbury Park                                                NaN   
4            Ancaster Park                                                NaN   

  Winter Play                                             Map It  \
0         NaN  ('27  Glen Watford Dr ', '43.7882598434', '-79...   
1         NaN  ('111  Alamosa Dr ', '43.7810883366', '-79.369...   
2         NaN  ('41  Deanlea Crt ', '43.7396441768', '-79.558...   
3         Yes  ('151 Culford Rd ', '43.7061864958', '-79.4834...   
4         NaN  ('43  Ancaster Rd ', '43.7334252351', '-79.465...   

          Phone    Type Lights  Courts  
0           311    Club    Yes       4  
1  416-496-0225    Clu

In [29]:
# prompt: List all courts with 5+ courts and has lights

def courts_with_lights_n_courts(df, n_courts=5, has_lights="Yes"):
    filtered_df = df[(df['Courts'] >= n_courts) & (df['Lights'] == has_lights)]
    return filtered_df

filtered_courts = courts_with_lights_n_courts(df)
# Select and print the 'Court Name' and 'Address' columns for the filtered courts
print("\nCourts with 5+ courts and lights:")
if not filtered_courts.empty:
    print(filtered_courts[['Location', 'Map It', 'Courts']])
else:
    print("No courts found matching the criteria.")


Courts with 5+ courts and lights:
                                              Location  \
11                             Kew Gardens Tennis Club   
13                                Bellbury Tennis Club   
19   Nyta Winter Tennis Club,The Don Mills Tennis Club   
35                                        Cosburn Park   
41                                      Eglinton Flats   
60                         Bayview Village Tennis Club   
62                              Heron Park Tennis Club   
63                                           High Park   
64                             Howard Park Tennis Club   
75                              Davisville Tennis Club   
78                      Scarborough Winter Tennis Club   
84                        Thorncliffe Park Tennis Club   
102                             Moore Park Tennis Club   
139                  Sir Winston Churchill Tennis Club   
157                                Leaside Tennis Club   
160                           Wanless

In [30]:
# prompt: Using python, find the courts nearest to me. The "Map It" column in the df has object ("Address", "Lat", "Long"). Use this to list all courts sorted by distance to me

import pandas as pd
!pip install haversine
from haversine import haversine, Unit

def get_nearest_courts(df, my_location):
  # Function to extract latitude and longitude from the 'Map It' string
  def extract_lat_lon(map_it_string):
      parts = map_it_string.strip("()").split(',')
      if len(parts) == 3:
          lat = float(parts[1].replace("'","").strip())
          lon = float(parts[2].replace("'","").strip())
          return lat, lon

  # Apply the function to create new 'Latitude' and 'Longitude' columns
  df[['Latitude', 'Longitude']] = df['Map It'].apply(lambda x: pd.Series(extract_lat_lon(x)))

  # Calculate the distance for each court to your location
  df['Distance'] = df.apply(lambda row: haversine(my_location, (row['Latitude'], row['Longitude']), unit=Unit.MILES) if pd.notnull(row['Latitude']) else float('inf'), axis=1)

  # Sort the DataFrame by distance
  df_sorted_by_distance = df.sort_values(by='Distance')
  return df_sorted_by_distance

# Assume your current location (latitude, longitude)
my_location = (43.6532, -79.3832) # Example: Toronto city center
df_sorted_by_distance = get_nearest_courts(df, my_location)

# Print the sorted list of courts (or a selection of columns)
print("\nAll courts sorted by distance from your location:")
print(df_sorted_by_distance[['Location', 'Distance']].head()) # Print the first 5 nearest courts



All courts sorted by distance from your location:
                         Location  Distance
103                     Moss Park  0.619680
143  Stanley Park South - Toronto  1.504611
158        Trinity Bellwoods Park  1.599219
120                  Ramsden Park  1.677393
122           Riverdale Park East  1.799809


In [31]:
sorted_by_distance_filtered = get_nearest_courts(filtered_courts, my_location)
print("\nCourts with 5+ courts and lights sorted by distance from your location:")
print(sorted_by_distance_filtered[['Location', 'Distance']])


Courts with 5+ courts and lights sorted by distance from your location:
                                              Location   Distance
139                  Sir Winston Churchill Tennis Club   2.418781
102                             Moore Park Tennis Club   2.785142
75                              Davisville Tennis Club   3.292309
157                                Leaside Tennis Club   3.703041
84                        Thorncliffe Park Tennis Club   3.767832
35                                        Cosburn Park   3.812685
64                             Howard Park Tennis Club   4.005749
63                                           High Park   4.005749
11                             Kew Gardens Tennis Club   4.326585
160                           Wanless Park Tennis Club   5.251274
172                                 Valley Tennis Club   6.131049
41                                      Eglinton Flats   6.207632
19   Nyta Winter Tennis Club,The Don Mills Tennis Club   6.635367
60 

<ipython-input-30-7e6ee889b0be>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Latitude', 'Longitude']] = df['Map It'].apply(lambda x: pd.Series(extract_lat_lon(x)))
<ipython-input-30-7e6ee889b0be>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Distance'] = df.apply(lambda row: haversine(my_location, (row['Latitude'], row['Longitude']), unit=Unit.MILES) if pd.notnull(row['Latitude']) else float('inf'), axis=1)
